In [2]:
from typing import NamedTuple
import re

import utils

## Day 7: Camel Cards

[#](https://adventofcode.com/2023/day/7) Camel cards is a game similar to poker, but a bit simpler. We get dealt multiple hands of 5 cards, and need to rank them in order of strength.

First up, dealing with how to rank cards by making a dict transforming each card to its rank:

In [37]:
cards = "A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, 2".split(", ")[::-1]
card_rank = {card: cards.index(card) + 2 for card in cards}
card_rank

{'2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'T': 10,
 'J': 11,
 'Q': 12,
 'K': 13,
 'A': 14}

Listing hand types in order of strength:

In [113]:
hand_orders = [
    "high card",
    "one pair",
    "two pair",
    "three of a kind",
    "full house",
    "four of a kind",
    "five of a kind",
]

Now to figure out what kind of hand type a hand is. I'm using sets here, but should be plenty of other ways to do this, e.g by counting characters. In actual poker, we would also check for flushes and straights, which is a bit harder.

In [125]:
def get_hand_type(hand):
    """get the type of a hand, i.e full house etc"""

    chars = set(hand)
    num_cards = len(chars)

    match num_cards:
        case 1:
            hand_type = "five of a kind"
        case 2:
            if min([hand.count(char) for char in chars]) >= 2:
                hand_type = "full house"
            else:
                hand_type = "four of a kind"
        case 3:
            # check for two pair
            if max([hand.count(char) for char in chars]) <3:
                hand_type = "two pair"
            else:
                hand_type = "three of a kind"
        case 4: 
            hand_type = "one pair"
        case 5:
            hand_type = "high card"
    return hand_type

get_hand_type("32T3K"), get_hand_type("T55J5"), get_hand_type("KTJJT")

('one pair', 'three of a kind', 'two pair')

I'm only parsing the input here, becuase for each hand type I want to store the order. This will make it easier later on to put all the cards in order, as we only have to compare the cards with the same order to each other, which makes the hand ranking problem much easier/faster than comparing each hand to every other hand.

In [146]:
test: str = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

inp = utils.get_input(7, splitlines=False)


class Hand(NamedTuple):
    hand: str
    bid: int
    order: int


def parse(inp=test, verbose=False):
    rounds = []
    for line in inp.strip().splitlines():
        hand, bid = line.split(" ")
        if verbose:
            print(hand, get_hand_type(hand))
        rounds.append(Hand(hand, int(bid), hand_orders.index(get_hand_type(hand))))

    return sorted(rounds, key=lambda hand: hand.order)


hands = parse(verbose=True)
hands

32T3K one pair
T55J5 three of a kind
KK677 two pair
KTJJT two pair
QQQJA three of a kind


[Hand(hand='32T3K', bid=765, order=1),
 Hand(hand='KK677', bid=28, order=2),
 Hand(hand='KTJJT', bid=220, order=2),
 Hand(hand='T55J5', bid=684, order=3),
 Hand(hand='QQQJA', bid=483, order=3)]

So now we have a a list of hands, sorted in order. So we now need to sort all the hands in each order. First up, a simple function to compare two hands:

In [197]:
def break_tie(hand, hand2):
    """returns the stronger hand, assuming two hands of same rank are passed"""
    for c1, c2 in zip(hand.hand, hand2.hand):
        if card_rank[c1] != card_rank[c2]:
            if card_rank[c1] > card_rank[c2]:
                return hand2, hand
            else:
                return hand, hand2
    return [hand, hand2]  # in case the two hands are identical


break_tie(hands[1], hands[2])
break_tie(hands[3], hands[4])

(Hand(hand='T55J5', bid=684, order=3), Hand(hand='QQQJA', bid=483, order=3))

To solve this, we need to compare each hand to every other hand and rank them. This is almost a search problem...

In [213]:
def sort_hands(hands):
    hands_sorted = []
    for order in range(len(hand_orders)):
        hands_in_order = [hand for hand in hands if hand.order == order]
        num_hands = len(hands_in_order)
        if num_hands > 0:
            print(f"Processing {num_hands} {hand_orders[order]} cards")
        if num_hands < 1:
            pass
        elif num_hands == 1:
            hands_sorted += hands_in_order
        else:
            for i in range(len(hands_in_order) - 1):
                hands_in_order[i : i + 2] = break_tie(
                    hands_in_order[i], hands_in_order[i + 1]
                )
            hands_sorted += hands_in_order

    return hands_sorted


sort_hands(hands)

Processing 1 one pair cards
Processing 2 two pair cards
Processing 2 three of a kind cards


[Hand(hand='32T3K', bid=765, order=1),
 Hand(hand='KTJJT', bid=220, order=2),
 Hand(hand='KK677', bid=28, order=2),
 Hand(hand='T55J5', bid=684, order=3),
 Hand(hand='QQQJA', bid=483, order=3)]

In [220]:
def solve(inp=test, verbose: bool = False):
    hands = parse(inp)
    print(f"-----Sorting {len(hands)} into order and returning total winnings-----")
    winnings = []
    for i, hand in enumerate(sort_hands(hands)):
        winnings.append((i + 1) * hand.bid)

    return sum(winnings)


# assert solve(test) ==  # example answer
assert solve() == 6440
solve(inp)

-----Sorting 5 into order and returning total winnings-----
Processing 1 one pair cards
Processing 2 two pair cards
Processing 2 three of a kind cards
-----Sorting 1000 into order and returning total winnings-----
Processing 186 high card cards
Processing 279 one pair cards
Processing 174 two pair cards
Processing 164 three of a kind cards
Processing 104 full house cards
Processing 92 four of a kind cards
Processing 1 five of a kind cards


248857361

## Part 2



In [11]:
def solve_2(inp=test, verbose: bool = False):
    data = parse(inp)

    return None


# assert solve_2(test) ==  # example answer
solve_2(inp)